# 🔥 Desafio - **Média histórica - Monitoramento hídrico**

## Enunciado:

- As tabelas **`parameter_sample`** e **`parameter_result`** armazenam dados de amostras e resultados, respectivamente, de monitoramento de qualidade hídrica.
- As amostras são derivadas dos pontos de monitoramento (tabela `station`) e os resultados são derivados das amostras. O esquema a seguir ilustra os relacionamentos entre as tabelas.

![station_samples_results](../../img/desafios/aula03/station_samples_result.png)

Duas análises comuns subsidiadas por estes dados são a evolução temporal dos parâmetros (avaliada normalmente por ferramentas estatísticas); e a avaliação de conformidade legal, através da comparação das medições a valores de referência.

## Objetivo

O desafio é escrever uma consulta SQL para identificar resultados de qualidade **acima da média histórica de cada parâmetro em pelo menos 20%**, ou seja, cada resultado deve ser comparado à média de todos os resultados do mesmo parâmetro, no mesmo ponto, anteriormente observados.

Os seguintes campos devem ser retornados pela consulta SQL:
- Codigo HGA: Nome HGA do ponto de monitoramento (station.Name)
- Local: Localização do ponto de monitoramento (station.zlocation)
- ID da amostra: Identificador da amostra de qualidade (parameter_sample.sample_id)
- Data da amostra: Data de amostragem (parameter_sample.sample_date)
- Parametro: Nome do parametro químico analisado (parameter_result.chemical_name)
- Resultado: Valor numérico obtido na análise do parâmetro (parameter_result.result_value)
- Unidade: Unidade referente ao parâmetro analisado (parameter_result.result_unit)
- **Média Histórica: Valor da média do monitoramento do parâmetro**
- **Flag: Identificação se o resultado obtido é, no mínimo, 20% superior à média histórica**

****

In [ ]:
-- Dica: Como comparar dinamicamente cada resultado apenas aos anteriores à ele ??

-- Utilizando uma subquery:
SELECT
    s.Name                      AS [Codigo HGA]
    ,s.zlocation                AS [Local]
    ,a.sample_id                AS [ID Amostra]
    ,a.sample_date              AS [Data Amostra]
    ,r.chemical_name            AS [Parametro]
    ,r.result_value             AS [Resultado]
    ,r.result_unit              AS [Unidade]

    ,(
        SELECT AVG(res.result_value)
        FROM station st
            INNER JOIN parameter_sample ams
                ON st.ID = ams.Station
            INNER JOIN parameter_result res
                ON ams.sample_id = res.sample_id
        WHERE
            st.ID = s.ID AND
            res.chemical_name = r.chemical_name AND 
            ams.sample_date < a.sample_date
    )                           AS [Media Hist]

FROM station s
    INNER JOIN parameter_sample a
        ON s.ID = a.Station
    INNER JOIN parameter_result r
        ON a.sample_id = r.sample_id
WHERE s.Name = 'WST-22' AND r.chemical_name = 'Alcalinidade Total'

In [ ]:
-- Utilizando a função OVER em conjunto com AVG
SELECT
    s.Name                      AS [Codigo HGA]
    ,s.zlocation                AS [Local]
    ,a.sample_id                AS [ID Amostra]
    ,a.sample_date              AS [Data Amostra]
    ,r.chemical_name            AS [Parametro]
    ,r.result_value             AS [Resultado]
    ,r.result_unit              AS [Unidade]
    ,AVG(r.result_value) OVER (PARTITION BY s.Name, r.chemical_name ORDER BY a.sample_date)  AS [Media Hist]
FROM station s
    INNER JOIN parameter_sample a
        ON s.ID = a.Station
    INNER JOIN parameter_result r
        ON a.sample_id = r.sample_id
WHERE s.Name = 'WST-22' AND r.chemical_name = 'Alcalinidade Total'

In [ ]:
WITH medias AS (
    SELECT
        s.Name                      AS [Codigo HGA]
        ,s.zlocation                AS [Local]
        ,a.sample_id                AS [ID Amostra]
        ,a.sample_date              AS [Data Amostra]
        ,r.chemical_name            AS [Parametro]
        ,r.result_value             AS [Resultado]
        ,r.result_unit              AS [Unidade]
        ,AVG(r.result_value) OVER (PARTITION BY s.Name, r.chemical_name ORDER BY a.sample_date)  AS [Media Hist]
    FROM station s
        INNER JOIN parameter_sample a
            ON s.ID = a.Station
        INNER JOIN parameter_result r
            ON a.sample_id = r.sample_id
    WHERE s.Name = 'WST-22' AND r.chemical_name = 'Alcalinidade Total'
)

SELECT
    *,
    Flag = IIF([Resultado] > 1.2 * [Media Hist], 'Acima', 'Abaixo')
FROM medias


# 🤯 Desafio desafiador

Ainda no contexto dos dados de monitoramento de qualidade hídrica, normalmente são utilizados relatórios/gráficos para melhor visualização e análise. Um exemplo de relatório, mostrado a seguir, considera a apresentação dos dados da amostragem em formato tabular e os parâmetros em formato aberto (nas colunas da matriz).

![Report](../../img/desafios/aula03/report.png)

Uma importante análise dos dados de qualidade hídrica se refere à saúde e consistência dos próprios dados (QAQC). Por exemplo, para alguns conjuntos de parâmetros, é esperada uma alta correlação entre os resultados. Um exemplo de comparação ocorre entre os parâmetros **Solidos Dissolvidos Totais (SDT)** e **Condutividade Eletrica**, em que considera-se aceitável uma razão SDT / CE na faixa de 55% - 75%.

## Objetivo

Criar uma consulta SQL para retornar uma análise de QAQC da razão entre os parâmetros **Solidos Dissolvidos Totais (SDT)** e **Condutividade Eletrica**.

O formato da tabela gerada deve ser o seguinte:
- Codigo HGA: Nome HGA do ponto de monitoramento (station.Name)
- ID da amostra: Identificador da amostra de qualidade (parameter_sample.sample_id)
- Solidos Dissolvidos Totais (SDT): Resultados médios do monitoramento de [Solidos Dissolvidos Totais (SDT)] para o ano/mês 
- Condutividade: Resultados médios do monitoramento de [Condutividade Eletrica] para o ano/mês
- SDT / CE: Razão entre o resultado de [Solidos Dissolvidos Totais (SDT)] e o da [Condutividade Eletrica], para análise do QAQC

PS.: O cálculo da razão faz sentido apenas para amostras com resultados **dos dois parâmetros**.

In [ ]:
WITH cte AS (
    SELECT
        s.Name                      AS [Codigo HGA]
        ,a.sample_id                AS [ID Amostra]
        ,r.chemical_name            AS [Parametro]
        ,r.result_value             AS [Resultado]
    FROM station s
        INNER JOIN parameter_sample a
            ON s.ID = a.Station
        INNER JOIN parameter_result r
            ON a.sample_id = r.sample_id
    WHERE 
        r.chemical_name IN ('Solidos Dissolvidos Totais (SDT)', 'Condutividade Eletrica')
)

SELECT
    [Codigo HGA]
    ,[ID Amostra]
    ,[Solidos Dissolvidos Totais (SDT)]
    ,[Condutividade Eletrica]
    ,[Razão %] = ROUND([Solidos Dissolvidos Totais (SDT)] / [Condutividade Eletrica] * 100, 2)
FROM cte
PIVOT (
    AVG([Resultado])
    FOR [Parametro] IN ([Solidos Dissolvidos Totais (SDT)] ,[Condutividade Eletrica])
) AS pivot_table
WHERE
    [Solidos Dissolvidos Totais (SDT)] IS NOT NULL
    AND [Condutividade Eletrica] IS NOT NULL